In [1]:
import h5py
import os
import numpy as np
import tensorflow as tf
import pandas as pd
import time

from dotenv import load_dotenv

from sklearn.model_selection import train_test_split

from my_functions.my_fun import *
from my_functions.my_models import vgg16,vgg16_svm
from my_functions.My_nn import My_nn

In [2]:
load_dotenv()

ASSETS_DIR = os.getenv('ASSETS_DIR')
ANOT_DIR = os.getenv('PMEMO_ANOTATIONS')
stat_anot_file = os.path.join(ANOT_DIR,'static_annotations.csv')
META_DIR = os.getenv('PMEMO_META')
aux_dir = META_DIR.split('.')[0]
META_DIR = f'{aux_dir}_total.csv'



# Cargar datos chromas magnitud

In [3]:
chromas_m = load_spectrograms(filename='chromas_magnitud')

In [4]:
chromas_m.shape

(767, 224, 224, 3)

# Preparar etiquetas

In [5]:
anot_df = pd.read_csv(stat_anot_file)
anot_df.head(10)

,musicId,Arousal(mean),Valence(mean)
0,1,0.4000,0.5750
1,4,0.2625,0.2875
2,5,0.1500,0.2000
3,6,0.5125,0.3500
4,7,0.7000,0.7250
5,8,0.3875,0.2250
6,9,0.4500,0.2875
7,10,0.4375,0.4250
8,12,0.6875,0.5750
9,13,0.8250,0.5875


In [6]:
labels = anot_df[['Arousal(mean)','Valence(mean)']].values
print(labels)
print(labels.shape)

[[0.4    0.575 ]
 [0.2625 0.2875]
 [0.15   0.2   ]
 ...
 [0.7125 0.6625]
 [0.875  0.775 ]
 [0.6625 0.575 ]]
(767, 2)


# Dividir conjunto de datos

In [7]:
pmemo_meta = pd.read_csv(META_DIR)
pmemo_meta.head(2)

,musicId,fileName,title,artist,album,duration,chorus_start_time,chorus_end_time
0,1,1.mp3,Good Drank,2 Chainz,"Def Jam Presents: Direct Deposit, Vol. 2",32.10,02:35,03:05
1,4,4.mp3,X Bitch (feat. Future),21 Savage,Savage Mode,28.09,03:00,03:26


In [8]:
indices = np.arange(len(pmemo_meta))
idx = np.random.randint(0,indices.shape[0])
print(idx)
print(pmemo_meta.iloc[idx])

591
musicId                                 757
fileName                            757.mp3
title                   Mary, Did You Know?
artist                           Pentatonix
album                That's Christmas To Me
duration                              38.12
chorus_start_time                     02:37
chorus_end_time                       03:13
Name: 591, dtype: object


In [9]:
# Mezcla los índices de manera aleatoria
np.random.shuffle(indices)

# Calcula los tamaños para cada conjunto
train_size = int(0.6 * len(indices))
val_size = int(0.2 * len(indices))
test_size = len(indices) - train_size - val_size  # Para asegurar que se cubre todo

# Divide el array en entrenamiento, validación y prueba
train_indices = indices[:train_size]
val_indices = indices[train_size:train_size + val_size]
test_indices = indices[train_size + val_size:]

print(f"Entrenamiento: {len(train_indices)}")
print(f"Validación: {len(val_indices)}")
print(f"Pruebas: {len(test_indices)}")

Entrenamiento: 460
Validación: 153
Pruebas: 154


In [10]:
X_train = []
y_train = []
for idx in train_indices:
    X_train.append(chromas_m[idx])
    y_train.append(labels[idx])

X_train = np.array(X_train)
y_train = np.array(y_train)

X_val = []
y_val = []
for idx in val_indices:
    X_val.append(chromas_m[idx])
    y_val.append(labels[idx])
X_val = np.array(X_val)
y_val = np.array(y_val)

X_test = []
y_test = []
for idx in test_indices:
    X_test.append(chromas_m[idx])
    y_test.append(labels[idx])

X_test = np.array(X_test)
y_test = np.array(y_test)

print(f'Datos de entrenamiento shape: {X_train.shape}')
print(f'Etiquetas de entrenamiento shape: {y_train.shape}')

print(f'Datos de pruebas shape: {X_test.shape}')
print(f'Etiquetas de pruebas shape: {y_test.shape}')

print(f'Datos de validación shape: {X_val.shape}')
print(f'Etiquetas de validación shape: {y_val.shape}')


Datos de entrenamiento shape: (460, 224, 224, 3)
Etiquetas de entrenamiento shape: (460, 2)
Datos de pruebas shape: (154, 224, 224, 3)
Etiquetas de pruebas shape: (154, 2)
Datos de validación shape: (153, 224, 224, 3)
Etiquetas de validación shape: (153, 2)


# Guardar datos procesados

In [11]:
# Guardar los datos en un archivo .h5
DATOS_DIR = os.path.join(ASSETS_DIR,'processed_chromas_m.h5')

with h5py.File(DATOS_DIR, 'w') as hf:
    hf.create_dataset('X_train', data=X_train)
    hf.create_dataset('X_test', data=X_test)
    hf.create_dataset('X_val', data=X_val)
    hf.create_dataset('y_train', data=y_train)
    hf.create_dataset('y_test', data=y_test)
    hf.create_dataset('y_val', data=y_val)

In [12]:
# X_train,X_test,y_train,y_test = train_test_split(chromas_m,
#                                                  labels,
#                                                  test_size=0.4,
#                                                  random_state=43)

# X_test,X_val,y_test,y_val = train_test_split(X_test,
#                                              y_test,
#                                              test_size=0.5,
#                                              random_state=43)


# print(f'Datos de entrenamiento shape: {X_train.shape}')
# print(f'Etiquetas de entrenamiento shape: {y_train.shape}')

# print(f'Datos de pruebas shape: {X_test.shape}')
# print(f'Etiquetas de pruebas shape: {y_test.shape}')

# print(f'Datos de validación shape: {X_val.shape}')
# print(f'Etiquetas de validación shape: {y_val.shape}')

# Datos de entrenamiento shape: (460, 244, 224, 3)
# Etiquetas de entrenamiento shape: (460, 2)
# Datos de pruebas shape: (153, 244, 224, 3)
# Etiquetas de pruebas shape: (153, 2)
# Datos de validación shape: (154, 244, 224, 3)
# Etiquetas de validación shape: (154, 2)



In [13]:
X_train.shape

(460, 224, 224, 3)

# VGG16

In [14]:
modelo_vgg = vgg16(trainable=True)

Model: "model"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 input_2 (InputLayer)        [(None, 224, 224, 3)]     0         
                                                                 
 tf.__operators__.getitem (S  (None, 224, 224, 3)      0         
 licingOpLambda)                                                 
                                                                 
 tf.nn.bias_add (TFOpLambda)  (None, 224, 224, 3)      0         
                                                                 
 vgg16 (Functional)          (None, 512)               14714688  
                                                                 
 flatten (Flatten)           (None, 512)               0         
                                                                 
 dense (Dense)               (None, 256)               131328    
                                                             

In [15]:
exp = My_nn(X_train=X_train,y_train=y_train,
            X_test=X_test,y_test=y_test,
            X_val=X_val,y_val=y_val,
            model=modelo_vgg)

Model: "model"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 input_2 (InputLayer)        [(None, 224, 224, 3)]     0         
                                                                 
 tf.__operators__.getitem (S  (None, 224, 224, 3)      0         
 licingOpLambda)                                                 
                                                                 
 tf.nn.bias_add (TFOpLambda)  (None, 224, 224, 3)      0         
                                                                 
 vgg16 (Functional)          (None, 512)               14714688  
                                                                 
 flatten (Flatten)           (None, 512)               0         
                                                                 
 dense (Dense)               (None, 256)               131328    
                                                             

c:\Users\leo_d\anaconda3\envs\mirtools\lib\site-packages\visualkeras\layered.py:86: UserWarning: The legend_text_spacing_offset parameter is deprecated and will be removed in a future release.
  warnings.warn("The legend_text_spacing_offset parameter is deprecated and will be removed in a future release.")


In [16]:
hist = exp.train()

Epoch 1/20
 8/29 [=======>......................] - ETA: 1:33 - loss: 16.5395 - mae: 2.3477

KeyboardInterrupt: 

In [ ]:
exp.show_loss_history(hist)

In [ ]:
loss,mae = exp.evaluate()
print(loss)
print(mae)

In [ ]:
y_pred = exp.recognize()
print(y_pred.shape)

In [ ]:
reporte = exp.get_report(y_true=y_test,y_pred=y_pred)
for k in reporte:
    print(f'{k}: {reporte[k]}')

In [ ]:
print(np.sort(y_test))

In [ ]:
print(np.sort(y_pred))

In [24]:
exp.model_name = 'vgg16_chroma_m'
exp.save_model()